<a href="https://colab.research.google.com/github/JonasRigo/AD-Numerical-Renormalization-Group/blob/main/dNRG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differentiable Numerical Renormalization Group for a single Anderson impurity model
by Jonas B. Rigo and Andrew K. Mitchell
 
In this notebook we follow the implementation of NRG for a single Anderson Impurity Model (siAM) laid out by *H. R. Krishna-murthy, J. W. Wilkins and K. G. Wilson* in *PRB 21, 3 1980*.
 
## Libraries
 
We use the `jax` library for efficient GPU backed implementation and easy automatic differentiation.

In [7]:
from jax.config import config # enable for 64 precision
config.update("jax_enable_x64",True) 

import jax
import jax.numpy as jp
from jax import ops as jops
import jax.scipy.linalg as la
from jax import jacrev, grad, jacfwd
from jax import make_jaxpr
from jax import custom_vjp
from jax import custom_jvp
from functools import partial

# Global variables
 
The NRG solver is based on iteratively expanding the Hilbert space of the Hamiltonian and then diagonalizing the grown Hamiltonian. The Hilbert space of the Hamiltonain at a step $N$ is always multiplied by the same $4$ dimensional Hilbert space of an $\uparrow$ and a $\downarrow$ flavoured fermions. The matrix elements of the grown Hamailtonian can be obtained as follows:
 
$\langle i',l',N,\vert H_{N+1}\vert i,l,N\rangle = \Lambda^{1/2} E(l,N)\delta_{i i'}\delta_{l l'}+ t_N(\langle l',N\vert f^\dagger_{N\sigma}\vert l,N\rangle \langle i' \vert f_{N+1\sigma}\vert i\rangle + \rm h.c. )$
 
where $l$ enumerates the eigenbasis of $H_N$ at iteration $N$ and $i$ enumerates the basis of a single Wilson chain site in the following way
 
$\vert i = 0 \rangle = \vert vac \rangle  \\ \vert i = 1 \rangle = f^{\dagger}_{\uparrow}\vert vac\rangle \\
\vert i = 2 \rangle = f^{\dagger}_{\uparrow}f^{\dagger}_{\downarrow}\vert vac\rangle \\
\vert i = 3 \rangle = f^{\dagger}_{\downarrow}\vert vac\rangle $
 
Since the basis of a single Wilson chain site never changes, we can introduce 
 
$\eta_{\sigma i i’} = \langle i' \vert f_{N+1\sigma}\vert i\rangle$,

which is saved as a global tensor called `elemaddedsite`. In the following we refer to `elemaddedsite` as “transfer tensor”.


In [8]:
elemaddedsite = jp.zeros((2,4,4))

elemaddedsite = jops.index_update(elemaddedsite,jops.index[0,0,3],1.)
elemaddedsite = jops.index_update(elemaddedsite,jops.index[0,1,2],-1.)
elemaddedsite = jops.index_update(elemaddedsite,jops.index[1,0,1],1.)
elemaddedsite = jops.index_update(elemaddedsite,jops.index[1,3,2],1.)

elemaddedsite_index = [[1,3,2],[1,0,1],[0,1,2],[0,0,3]]

print("elemaddedsite: \n", elemaddedsite)

elemaddedsite: 
 [[[ 0.  0.  0.  1.]
  [ 0.  0. -1.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  1.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  1.  0.]]]


# Hamiltonian class
 
The Hamiltonain has to undergo two important steps:
 
* Initialize: set up $H_{-1} = H_{imp}$
* Grow: $H_N \rightarrow H_{N+1}$
 
For the *initialization* we need to define the impurity Hamiltonian (includes no bath or hybridization) and bring it in diagonal form. For the siAM the occupation basis is already an eigenbasis.
 
To *grow* the Hamiltonian means to add a Wilson chain site to the Hamiltonian
$\langle i',l',N,\vert H_{N+1}\vert i,l,N\rangle = \Lambda^{1/2} E(l,N)\delta_{i i'}\delta_{l l'}+ t_N(\langle l',N\vert f^\dagger_{N\sigma}\vert l,N\rangle \langle i' \vert f_{N+1\sigma}\vert i\rangle + \rm h.c. )$,
 
where $t_N$ is the Wislon chain hopping and $\vert i,l,N\rangle$ an eigenvector of $H_N$. This equation can be rewritten as
$\langle i',l',N,\vert H_{N+1}\vert i,l,N\rangle = \Lambda^{1/2} E(l,N)\delta_{i i'}\delta_{l l'}+ t_N(\eta^{N+1}_{\sigma l l’} \eta_{\sigma i’ i} + \rm h.c. )$,
 
where $\eta^{N+1}_{\sigma l l’}$ is the complex transposed transfer tensor in the eigenbasis of $H_N$. How to obtain the transfer tensor in the basis of $H_N$ will be explained in detail later. The last crucial step performed in `grow` is to truncate the eigenbasis of $H_N$. No more than `rlim` states are kept.

In [9]:
class Hamiltonian(object):
  def __init__(self, eps, U, V,lmax=40,rlim=200,Lam=3.,Himp_dim=4):
    """
    Initialize the Hamiltonian class with all impurity couplings and the
    parameters required by the NRG.
    ===========================================================
    input:
    eps: on-site energy
    U: on-site interaction
    V: hybridization strength
    lmax: chain length
    rlim: maximum number of kept states
    Lam: discretization parameter
    Himp_dim: dimension of a Wilson chain site
    """
    print("Anderson impurity model")
    print("Lambda = ",Lam)
    print("max number of states kept at each iteration = ",rlim)
    print("\n")
    self.Lam = Lam
    self.rlim = rlim
    self.dim = Himp_dim
    self.aux_dim = 4
    self.eps = eps
    self.U = U
    self.V = V

  def initialize(self):
    """
    Create the `-1` Hamiltonian, it contains only the impurity without hybridization.
    """

    print("U/D = ", self.U)
    print("epsilon/D = ",self.eps)
    print("V/D = ",self.V)
    print("\n")

    """Renormalize the impurity parameters"""
    alambda = (self.Lam + 1.)/(self.Lam - 1.)*jp.log(self.Lam)/2. # this factor accounts for the discretation
    self.eps /= self.Lam
    self.U /= self.Lam
    
    """
    The impurity Hamiltonian is diagonal in the occupation basis. 
    We exploit that to write it in terms of its eigenvalues and use the occupation
    basis as eigenbasis.
    """
    energies = jp.zeros(self.dim)
    energies = jops.index_update(energies,jp.array([0,1,2,3]),[0.,self.eps,(2.*self.eps + self.U),self.eps]) 
    """
    The first hopping term is the hybridization between impurity and bath.
    """
    self.wilson_t = jp.sqrt(alambda/self.Lam) * self.V # * we account for the descrete bath
    """
    The transfer tensor in the basis of H_N becomes `elemlastsite`.
    """
    elemlastsite = jp.zeros((2,self.aux_dim,self.aux_dim))
    elemlastsite = jops.index_update(elemlastsite,0,jp.transpose(elemaddedsite[0]))
    elemlastsite = jops.index_update(elemlastsite,1,jp.transpose(elemaddedsite[1]))

    return energies, elemlastsite

  def grow(self,wilson_site,energies,elemlastsite):
    """
    The Hilbert space grows by an up and a down spin. We keep the
    old dimension locally in `dim` and update the dimension in `self` by
    multiplying it with the dimension of the added Hilbert space
    ===========================================================
    input:
    wilson_site: integer number defining the number of sites attached to the impurity
    energies: array containing eigenvalues of the Hamiltonian
    """

    dim = self.dim

    """
    This step is crucial: we limit the maximum amount of states by truncating the
    dimension of the Hilbert space.
    """
    self.dim = self.dim*self.aux_dim

    H = jp.zeros((self.dim,self.dim))

    """
    The diagonal consists only of the beforehand calculated energies. Mind the
    tesnor product! The factor `0.5` accounts for a factor `2` that the diagonal acquires
    after summing up the trasnpose of `H`
    """
    id = lambda x: dim*x

    for i in jp.arange(4):  
      H = jops.index_update(H,jops.index[id(i):id(i+1),id(i):id(i+1)], 0.5 * jp.diag(jp.sqrt(self.Lam) * energies[0:dim]))

    """
    We exploit the tensor nature of of `elementlastside` and `elementaddedstite` and 
    express the sum over the flavour (in this case spin) as a scalar product. Note that 
    this corresponds to taking a kronecker tensorproduct of the type: |N,l> x |i>.
    """
    for idx in elemaddedsite_index:
      sign, k, kp = idx
      H = jops.index_update(H,jops.index[id(k):id(k+1),id(kp):id(kp+1)],self.wilson_t * elemlastsite[sign]*elemaddedsite[sign,k,kp] * (-1.)**(k))

    """
    Add the hermitian conjugate that was neglected earlier
    """
    H += H.T

    """
    The wilson chain has now grown, so we calculate the next hoping paramter that links the 
    new Hamiltonian to next Hilbert space in the following iteration.
    """
    self.wilson_t = .5*(1.+self.Lam**(-1.))*(1.-self.Lam**(-wilson_site-1.))*((1.-self.Lam**(-2.*wilson_site-1.))*(1.-self.Lam**(-2.*wilson_site-3.)))**(-.5)

    return H

## Differentiable Hamiltonian class
 
Also the differentiable Hamitlonian has two functions
 
* Initialize
* Grow
 
For the initialization we need to define the derivative of the impurity Hamiltonian with respect to a coupling parameter.
 
To grow the Hamiltonian means in this case to propagate it forward. Where in the `Hamiltonian` routine we add sites to Hamiltonian, here we simply expand the Hilbert space and transform the differentiated Hamiltonian in the eigenbasis of the Hamiltonian obtained in `Hamiltonian`.

In [10]:
class dHamiltonian(object):
  def __init__(self, eps, U, V,lmax=40,rlim=200,Lam=3.,Himp_dim=4):
    """
    Initialize the Hamiltonian class with all impurity couplings and the
    parameters required by the NRG.
    ===========================================================
    input:
    eps: on-site energy
    U: on-site interaction
    V: hybridization strength
    lmax: chain length
    rlim: maximum number of kept states
    Lam: discretization parameter
    Himp_dim: dimension of a Wilson chain site
    """
    print("Anderson impurity model")
    print("Lambda = ",Lam)
    print("max number of states kept at each iteration = ",rlim)
    print("\n")
    self.Lam = Lam
    self.rlim = rlim
    self.dim = Himp_dim
    self.aux_dim = 4
    self.eps = eps
    self.U = U
    self.V = V
    self.lmax = lmax

  def initialize(self):
    """
    Here we create the derivitative of the `-1` Hamiltonian.
    """

    print("U/D = ", self.U)
    print("epsilon/D = ",self.eps)
    print("V/D = ",self.V)
    print("\n")

    """Renormalize the impurity parameters"""
    alambda = (self.Lam + 1.)/(self.Lam - 1.)*jp.log(self.Lam)/2. # this factor accounts for the discretation error
    self.eps /= self.Lam
    self.U /= self.Lam
    
    """
    The impurity Hamiltonian is diagonal in the occupation basis. 
    We exploit that to write it in terms of its eigenvalues and use the occupation
    basis as eigenbasis.
    """
    energies = jp.zeros(self.dim)
    energies = jops.index_update(energies,jp.array([0,1,2,3]),[0.,self.eps,(2.*self.eps + self.U),self.eps]) 
    self.ham = jp.diag(energies)

    """
    The first hopping term is the hybridization between impurity and bath:
    """
    self.wilson_t = jp.sqrt(alambda/self.Lam) * self.V


  def grow(self,eigsrset,rkept,trafoset):
    """
    The Hilbert space grows now for an up and a down spin. We keep the
    old dimension locally in `dim` and update the dimension in `self` by
    multiplying it with the dimension of the added Hilbert space
    ===========================================================
    input:
    eigsrset: list of oll eigenstates of all iterative diagonalization of the
    Hamiltonian.
    rkept: number of kept states in every iteration
    trafoset: list containing the `elemaddedsite` tensor in the eigenbasis of
    of all iterations of the Hamiltonian.
    """

    for i in range(self.lmax):
      print("i: ", i)
      eigs = eigsrset[i]
      dim = rkept[i]
      elemlastsite = trafoset[i]
      print("dim: ",dim)

      self.dim = rkept[i]*self.aux_dim

      H = jp.zeros((self.dim,self.dim))

      id = lambda x: dim*x

      self.ham = jp.dot(eigs.T,jp.dot(self.ham,eigs))
      for i in jp.arange(4):  
        H = jops.index_update(H,jops.index[id(i):id(i+1),id(i):id(i+1)], 0.5 * jp.sqrt(self.Lam) * self.ham[0:dim,0:dim])

      for idx in elemaddedsite_index:
        sign, k, kp = idx
        H = jops.index_update(H,jops.index[id(k):id(k+1),id(kp):id(kp+1)],self.wilson_t * elemlastsite[sign]*elemaddedsite[sign,k,kp] * (-1.)**(k))

      H += H.T
      self.ham = H
      
      """
      The wilson chain paramter is set to 0 becuase it vanishes through the derivative.
      """
      self.wilson_t = 0.

    return H

## Solver
 
The solver handles two main routines
 
 
*  Basis change of the transfer tensor
*  Diagonalization of the Hamiltonian
*  Computing the thermodynamics
 
### Transfer tensor
 
The basis change of the transfer tensor is a clever way to avoid high computational cost. We wrote the transfer tensor as follows
$ \eta_{\sigma i j} = \langle j \vert f_\sigma\vert i\rangle $, now we express it in the basis of $H_N$. Let $\vert i \rangle \otimes \vert l, N \rangle$ be the basis of $H_N$ and $U$ the unitary transformation that diagonalizes $H_{N+1}$. Then
 
$ \eta^{N+1}_{\sigma i j} = \eta_{\sigma i j} \sum_l U^\dagger \vert i \rangle \otimes \vert l, N \rangle \langle l, N \vert \otimes \langle j \vert U \\
\Rightarrow \eta^{N+1}_{\sigma i j} = \eta_{\sigma i j} U^\dagger \vert i \rangle \langle j \vert \otimes \mathbb{I}_l U$
 
and we can simplify the calculation of the transfer tensor to
$ \eta^{N+1}_{\sigma i j} = \eta_{\sigma i j} \left\vert \begin{matrix} U_{0j} \\ U_{1j} \\ U_{2 j} \\ U_{3 j} \end{matrix} \right\rangle
\otimes  \langle U_{i0} ~ U_{i1} ~ U_{i2} ~ U_{i 3} \vert$,

with $U_{ij} = \langle i \vert U \vert j \rangle$.

In [11]:
def transfertensor(eigen_system,dim,trunc_dim):
  """
  We use the knowledge about the transfer tensor `elemaddedsite` and calculate it
  in the eigenbasis of the Hamitlonian. Thus, conceptually we transform
  `elemaddesite` into `elemlastsite`.
  """
  id = lambda x: dim*x

  elemlastsite = jp.zeros((2,trunc_dim,trunc_dim))

  for idx in elemaddedsite_index:
    sigma, k, kp = idx
    elemlastsite = jops.index_add(elemlastsite,jops.index[sigma],elemaddedsite[sigma,k,kp]*jp.matmul(jp.transpose(eigen_system[id(k):id(k)+dim,0:trunc_dim]),eigen_system[id(kp):id(kp)+dim,0:trunc_dim]))
  return elemlastsite


### Thermodynamics
 
To extract some physical insight from the simulation we compute thermodynamic quantities. Since we have the diagonalized Hamiltonian from NRG it is straightforward to calculate the full density matrix
 
$\rho = \frac{1}{Z}e^{-\beta H_N},~Z=\sum_i e^{-\beta \lambda_i}$,

where $\lambda_i$ are the eigenvalues of $H_N$. The temperature in this definition comes from the Hamiltonian itself. In the NRG scheme one can relate a certain chain length $N$ to a temperature $T$. The inverse temperature is given defined as
$ \beta_N = 1/T_N$ 
 
$\beta_N \Lambda^{-(N-1)/2} = \bar{\beta},~\bar{\beta} \propto \mathcal{O}(1)$
 
For $\bar{\beta}$ we choose $\bar{\beta} = 0.9$. With this we know at which temperature the density matrix is calculated. Now we can compute is the system entropy 
 
$F = -T \ln(Z) \\
S = - \frac{\partial F}{\partial T} \\
\Rightarrow S = \bar{\beta}\langle H_N \rangle + \ln(Z),~\rm{with}~\langle H_N \rangle = \rm{tr}[\rho H ]$

In [12]:
def thermo(i,energies,Lam):
  """
  We calculate the density matrix for the momentary temperature
  =====================================================
  input:
  i: wilson chain length
  energies: eigen energies of the Hamiltonian
  Lam: discretization parameter
  """
  beta_bar = 0.9

  """the temperature is realted to the lenght of the wilson chain"""
  temperature = np.power(Lam,-0.5*(i - 1.))/beta_bar
  print("Temperature = ", temperature)

  """
  from the diagonal hamiltonian we can directly calcualte the
  partition funciton, desnity matrix and entropy
  """
  rho = np.exp(-beta_bar*energies) # desnity vector -> diagonal only
  exp_H = beta_bar*np.dot(energies,rho) # expectation value of the Hamiltonian
  Z = np.sum(rho) # partition function
  entropy = exp_H/Z + np.log(Z) # entropy

  return temperature, entropy

### Solver class

The iterative diagonalization routine.

In [13]:
class Solver(object):
  def __init__(self,ham,lmax):
    """
    input:
    ham: initialized hamiltonian of class type `Hamiltonian`
    lmax: maximum chain length
    """
    self.ham = ham
    self.lmax = lmax
    self.eigsrset = []
    self.rkept = []
    self.trafoset = []
    self.rseed = rseed

  def solve(self):
    energies, elemlastsite = self.ham.initialize()
    eigs = jp.eye(4,dtype=float)
    entropy = []
    
    for i in jp.arange(self.lmax):
      """
      We save the dimension before growing, then pass it to the 
      trans fertensor and truncating: 
      """
      dim = self.ham.dim 
      self.eigsrset += [eigs]
      self.trafoset += [elemlastsite]
      self.rkept += [self.ham.dim]
           
      """ Update for user: """
      print("Interation: ", i)
      print("States kept at this iteration = ", dim)

      """ iterate the wilson chain and diagonalize the Hamiltonian"""
      ham = self.ham.grow(i,energies,elemlastsite)
      energies, eigs = la.eigh(ham,turbo=True)
      """ set the groundstate to zero energy with a variable shift """
      #energies -= energies[0]

      """ update dimension and truncate """
      self.ham.dim = jp.minimum(self.ham.dim, self.ham.rlim)
      elemlastsite = transfertensor(eigs,dim,self.ham.dim)
      print("\n")

    self.eigsrset += [eigs]
    self.trafoset += [elemlastsite]
    self.rkept += [self.ham.dim]

    print("Calculation complete.")

    return energies, eigs

## Differentiable Solver Routine
 
Other than the non-differentiable solver routine `Solver` the differentiable solver routine does not return the eigenstates and eigenenergies. The `dSolver` routine returns the final Hamiltonian matrix $H_N$.

In [14]:
class dSolver(object):
  def __init__(self,ham,lmax,rseed):
    """
    input:
    ham: initialized hamiltonian of class type Hamiltonian
    lmax: maximum chain length
    rseed: numpy array containing ranodm values
    """
    self.ham = ham
    self.lmax = lmax
    self.eigsrset = []
    self.rkept = []
    self.trafoset = []
    self.rseed = rseed

  def solve(self):
    energies, elemlastsite = self.ham.initialize()
    eigs = jp.eye(4,dtype=float)
    entropy = []

    for i in jp.arange(self.lmax):

      dim = self.ham.dim 
      """ 
      Saving the eigensystem, the `elemaddedsite` tensor in the Hamiltonian 
      eigenbasis and the number of kept states.
      """
      self.eigsrset += [eigs]
      self.trafoset += [elemlastsite]
      self.rkept += [self.ham.dim]
           
      """ Update for user:"""
      print("Interation: ", i)
      print("States kept at this iteration = ", dim)

      """ iterate the wilson chain and diagonalize the Hamiltonian"""
      ham = self.ham.grow(i,energies,elemlastsite)
      """ we add random noise to the diagonal to lift the degeneracy of eigenvalues """
      energies, eigs = la.eigh(ham+jp.diag(rseed[:ham.shape[0]]),turbo=True)
      """ set the groundstate to zero energy with a variable shift """
      #energies -= energies[0]

      """ updated dimension and truncate """
      self.ham.dim = jp.minimum(self.ham.dim, self.ham.rlim)
      elemlastsite = transfertensor(eigs,dim,self.ham.dim)
      print("\n")

    self.eigsrset += [eigs]
    self.trafoset += [elemlastsite]
    self.rkept += [self.ham.dim]

    print("Calculation complete.")

    return ham

## Automatic Differentiation
 
### Differentiable `jax` NRG primitive
 
The derivative of the whole NRG code with respect to impurity coupling constants, is given through the derivative of the Hamiltonian $H_N$. The the derivative of $H_N$ with respect to impurity coupling constants can be obtained through propagating forward $\text{d} H_{\rm imp}$, such that it can act on the $N^{\rm th}$ Hilbert space associated to $H_N$.

In [15]:
@custom_jvp
def dH(eps,U,V,l,rlim,Lam):
    """
    Initialize the Hamiltonian and obtain the Hamiltonian matrix
    of maximal length.
    Note: Derivatives can only be obtained with resepect to
    eps, U or V.
    ===========================================================
    input:
    eps: on-site energy
    U: on-site interaction
    V: hybridization strength
    l: chain length
    rlim: maximum number of kept states
    Lam: discretization parameter
    """

    H1 = Hamiltonian(eps,U,V,l,rlim,Lam)
    S = dSolver(H1,l,rseed)
    return S.solve()

@dH.defjvp
def dH_jvp(primals, tangents):
    """
    Returns the derivative of the Hamiltonian with respect to the 
    input parameters `eps`, `U` or `V`.
    """
    eps_dot, U_dot, V_dot,adot,bdot,cdot = tangents
    eps,U,V,l,rlim,Lam = primals
    H1 = Hamiltonian(eps,U,V,l,rlim,Lam)
    S = dSolver(H1,l,rseed)
    ham = S.solve()

    """derivative wrt eps """
    H = dHamiltonian(1.,0.,0.,l,rlim,Lam)
    H.initialize()
    dH1 = H.grow(S.eigsrset,S.rkept,S.trafoset)

    """derivative wrt U """
    H = dHamiltonian(0.,1.,0.,l,rlim,Lam)
    H.initialize()
    dH2 = H.grow(S.eigsrset,S.rkept,S.trafoset)

    """derivative wrt V """
    H = dHamiltonian(0.,0.,1.,l,rlim,Lam)
    H.initialize()
    dH3 = H.grow(S.eigsrset,S.rkept,S.trafoset)

    """ accumulating derivative """
    primal_out = ham
    tangent_out = dH1*eps_dot + U_dot*dH2 + V_dot*dH3 + 0.*adot+0.*bdot+0.*cdot
    return primal_out, tangent_out

### Examples of possible derivatives

In [16]:
import numpy as np
""" 
The random values generated here are used the lift the 
degeneracies of the Hamiltonian. This allows the computation of the 
derivatives of the eigenvalues and eigenvectors.
"""
rseed = np.random.randn(4000)*1e-8

In [17]:
def free_energy(eps,U,V,l,rlim,Lam):
  """
  Calculate the free energy of an Anderson impurity model
  ===========================================================
  input: 
  eps: on-site energy
  U: on-site interaction
  V: hybridization strength
  l: chain length
  rlim: maximum number of kept states
  Lam: discretization parameter
  """
  ham = dH(eps,U,V,l,rlim,Lam)
  energies, eivecs = la.eigh(ham + jp.diag(rseed[:ham.shape[0]]),turbo=True)
  gs = energies[0]
  energies -= energies[0]
  beta_bar = 0.9
  rho = jp.exp(-energies*beta_bar)
  F = -(jp.log(jp.sum(rho))-beta_bar*gs)/beta_bar
  return F

def nexp(eps,U,V,l,rlim,Lam):
  """
  Calculate the occupation of an Anderson impurity model.
  ===========================================================
  input: 
  eps: on-site energy
  U: on-site interaction
  V: hybridization strength
  l: chain length
  rlim: maximum number of kept states
  Lam: discretization parameter
  """
  H1 = Hamiltonian(eps,U,V,l,rlim,Lam)
  S = Solver(H1,l)
  eivals, eivecs = S.solve()
  eivals -= eivals[0]

  """ obtaining the occupation operator """
  H = dHamiltonian(1.,0.,0.,l,rlim,Lam)
  H.initialize()
  dH1 = H.grow(S.eigsrset,S.rkept,S.trafoset)
  beta_bar = 0.9

  """ constructing the density matrix """
  rho = np.dot(eivecs,np.dot(np.diag(np.exp(-eivals*beta_bar)),eivecs.conj().T))

  """ calculating the thermodynamic expectation value """
  nrho = np.trace(np.dot(rho,dH1))/jp.trace(rho)
  return Lam**(-(l-2.)/2.)*nrho

""" backwards derivative of the free energy with respect to eps and U """
expn = jacrev(free_energy,[0,1])

Here we calculate $\partial_\epsilon F$, where $F$ is the free energy and $\epsilon$ the on-site energy.

In [18]:
eps,U,V,l,rlim,Lam = -0.15,0.3,0.1,2,200,3.
""" finite difference value to approximate the derivative """
dfs = 1e-6
""" finite difference derivative """
a = (free_energy(eps+dfs,U,V,l,rlim,Lam)/dfs-free_energy(eps,U,V,l,rlim,Lam)/dfs)
""" forward pass of the free energy """
b = free_energy(eps,U,V,l,rlim,Lam)
""" backward pass of the free energy """
c = expn(eps,U,V,l,rlim,Lam)[0]
""" thermodynamic expectation value of the occupation """
d = nexp(eps,U,V,l,rlim,Lam)

print("\n")
print("free energy: ",b)
print("\n")
print("automatic derivative: ",Lam**(-(l-2.)/2.)*b)
print("\n")
print("finite difference derivative: ",Lam**(-(l-2.)/2.)*a)
print("\n")
print("expectation value: ",d)

Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.3
epsilon/D =  -0.149999
V/D =  0.1


Interation:  0
States kept at this iteration =  4


Interation:  1
States kept at this iteration =  16


Calculation complete.
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.3
epsilon/D =  -0.15
V/D =  0.1


Interation:  0
States kept at this iteration =  4


Interation:  1
States kept at this iteration =  16


Calculation complete.
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.3
epsilon/D =  -0.15
V/D =  0.1


Interation:  0
States kept at this iteration =  4


Interation:  1
States kept at this iteration =  16


Calculation complete.
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.3
epsilon/D =  -0.15
V/D =  0.1


Interation:  0
States kept at this iteration =  4


Interation:  1
States kept 

Here we calculate $\frac{\partial^2}{\partial\epsilon \partial U} F$, where $F$ is the free energy and $\epsilon$ the on-site energy and $U$ the on-site interaction.

In [19]:
""" calculating the second order derivative of the free energy"""
dFdedU = jacrev(jacfwd(free_energy,[0]),[1])

In [20]:
""" finite difference value to approximate derivative """
dfs = 0.00390625
print("AD")
a = dFdedU(eps,U,V,l,rlim,Lam)
print("\n")
print("FD")
""" finite difference derivative """
b = nexp(eps,U+dfs,V,l,rlim,Lam)/dfs-nexp(eps,U,V,l,rlim,Lam)/dfs
print("\n")
print("automatic derivative: ",Lam**(-(l-2.)/2.)*a[0][0])
print("finite difference derivative: ",b)

AD
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  Traced<ConcreteArray(0.3, weak_type=True)>with<JVPTrace(level=2/0)>
  with primal = Traced<ConcreteArray(0.3, weak_type=True):JaxprTrace(level=1/0)>
       tangent = Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=1/0)>
epsilon/D =  -0.15
V/D =  0.1


Interation:  0
States kept at this iteration =  4


Interation:  1
States kept at this iteration =  16


Calculation complete.
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.0
epsilon/D =  1.0
V/D =  0.0


i:  0
dim:  4
i:  1
dim:  16
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  1.0
epsilon/D =  0.0
V/D =  0.0


i:  0
dim:  4
i:  1
dim:  16
Anderson impurity model
Lambda =  3.0
max number of states kept at each iteration =  200


U/D =  0.0
epsilon/D =  0.0
V/D =  1.0


i:  0
dim:  4
i:  1
dim:  16


FD
Anderson impurity